# Finance & Procurement: Budget Forecasting and Automated Offer Evaluation

In [ ]:
import pandas as pd, numpy as np
from sklearn.linear_model import LinearRegression
budget = pd.read_csv('budget_history.csv', parse_dates=['month'])
budget['t'] = range(len(budget))
X = budget[['t']].values
y = budget['actual_spend_usd'].values
model = LinearRegression().fit(X, y)
future = pd.DataFrame({'t': range(len(budget), len(budget)+6)})
future['month'] = pd.date_range(budget['month'].max() + pd.offsets.MonthEnd(1), periods=6, freq='M')
future['forecast_spend_usd'] = model.predict(future[['t']])
print(future[['month','forecast_spend_usd']])

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,3))
plt.plot(budget['month'], budget['actual_spend_usd'], label='Actual')
plt.plot(budget['month'], model.predict(budget[['t']]), label='Trend (LR)')
plt.plot(future['month'], future['forecast_spend_usd'], label='Forecast')
plt.legend(); plt.title('Budget Forecast'); plt.xlabel('Month'); plt.ylabel('USD'); plt.show()

## Offer Evaluation (Multi-criteria scoring)

In [ ]:
offers = pd.read_csv('procurement_offers.csv')
def normalize(series, invert=False):
    s = (series - series.min())/(series.max()-series.min())
    return 1 - s if invert else s
offers['score'] = (
    0.45*normalize(offers['price_usd'], invert=True) +
    0.25*normalize(offers['delivery_days'], invert=True) +
    0.10*offers['warranty_ge_2yrs'] +
    0.20*normalize(offers['supplier_score'])
)
top = offers.sort_values('score', ascending=False).head(10)
top